In [1]:
import torch
import torch.optim as optim
from torch.nn import functional as F
import torch.nn as nn
import torchvision

In [2]:
# [ch_out,ch_in]

w1, b1 = torch.randn(200, 784, requires_grad=True),\
         torch.zeros(200, requires_grad=True)
w2, b2 = torch.randn(200, 200, requires_grad=True),\
         torch.zeros(200, requires_grad=True)
w3, b3 = torch.randn(10, 200, requires_grad=True),\
         torch.zeros(10, requires_grad=True)

torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)

tensor([[-0.0081,  0.0698,  0.1792,  ...,  0.1079,  0.0630, -0.1812],
        [ 0.1557, -0.1281, -0.1360,  ...,  0.1373, -0.1511, -0.1252],
        [ 0.1557, -0.1146,  0.0860,  ..., -0.0055,  0.1864,  0.0501],
        ...,
        [-0.2130, -0.1637,  0.0208,  ..., -0.1124,  0.1164, -0.1571],
        [ 0.0456,  0.3765,  0.1294,  ...,  0.0869,  0.0239, -0.1952],
        [-0.0639, -0.0547,  0.1445,  ..., -0.0347, -0.0347, -0.0056]],
       requires_grad=True)

In [3]:
def forward(x):
    x = x@w1.t() + b1
    x = F.relu(x)
    x = x@w2.t() + b2
    x = F.relu(x)
    x = x@w3.t() + b3
    x = F.relu(x)     # logits
    
    return x

In [4]:
learning_rate = 0.01
epochs = 1
batch_size = 64

In [5]:
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data',
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),                       # 数据类型转化
                torchvision.transforms.Normalize((0.1307, ), (0.3081, )) # 数据归一化处理
    ])), batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data/',
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307, ), (0.3081, ))
    ])),batch_size=batch_size,shuffle=False)

In [6]:
optimizer = optim.SGD([w1,b1,w2,b2,w3,b3], lr = learning_rate)
criteon = nn.CrossEntropyLoss()

for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        data = data.view(-1,28*28)
        logits = forward(data)
        loss = criteon(logits,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = forward(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 3.031062
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.814657
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.586956
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.863106
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.699339
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.556181
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.614518
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.725399
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.261788
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.244436

Test set: Average loss: 0.0041, Accuracy: 9231/10000 (92%)

